# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [208]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm

import pylab as pl
import numpy as np


In [209]:
df_raw = pd.read_csv("../DAT-NYC-39/projects/unit-projects/project-3/assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [210]:
# frequency table for prestige and whether or not someone was admitted
print df.admit.value_counts()
print df.prestige.value_counts()


0    271
1    126
Name: admit, dtype: int64
2.0    148
3.0    121
4.0     67
1.0     61
Name: prestige, dtype: int64


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [211]:
dummy_ranks = pd.get_dummies(df.prestige,prefix='prestige')
dummy_ranks.columns = ['prestige_1', 'prestige_2', 'prestige_3', 'prestige_4']


#### 2.2 When modeling our class variables, how many do we need? 



Answer: Six.  admittance (the result), gpa, gre, and n-1 dummy variables for prestige.

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [212]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
print handCalc.head()

   admit    gre   gpa  prestige_1  prestige_2  prestige_3  prestige_4
0      0  380.0  3.61         0.0         0.0         1.0         0.0
1      1  660.0  3.67         0.0         0.0         1.0         0.0
2      1  800.0  4.00         1.0         0.0         0.0         0.0
3      1  640.0  3.19         0.0         0.0         0.0         1.0
4      0  520.0  2.93         0.0         0.0         0.0         1.0


In [213]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
pd.crosstab(df['admit'], df['prestige'], rownames=['admit'])

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,28,95,93,55
1,33,53,28,12


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [214]:
p = 33.0 / 28.0
print p

1.17857142857


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [215]:
q = 93.0 / 243.0
print q

0.382716049383


#### 3.3 Calculate the odds ratio

In [216]:
print p / q

3.07949308756


#### 3.4 Write this finding in a sentence: 

Answer: The odds ratio > 1 therefore attendance at a prestigiously-ranked school is positively associated with admittance.

#### 3.5 Print the cross tab for prestige_4

In [217]:
cols_to_keep = ['admit', 'gre', 'gpa']
df_dummies = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])

pd.crosstab(df_dummies['admit'], df_dummies['prestige_4'])

prestige_4,0.0,1.0
admit,,
0,216,55
1,114,12


#### 3.6 Calculate the OR 

In [218]:
odds_ratio = (12.0/55) / (114.0/216)
print odds_ratio

0.413397129187


#### 3.7 Write this finding in a sentence

Answer:  The odds ratio < 1 therefore attendance at a non-prestigiously-ranked school is not positively associated with admittance.

## Part 4. Analysis

In [219]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
print data.head()

   admit    gre   gpa  prestige_2  prestige_3  prestige_4
0      0  380.0  3.61         0.0         1.0         0.0
1      1  660.0  3.67         0.0         1.0         0.0
2      1  800.0  4.00         0.0         0.0         0.0
3      1  640.0  3.19         0.0         0.0         1.0
4      0  520.0  2.93         0.0         0.0         1.0


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [220]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [221]:

train_cols = data.columns[1:]

print list(train_cols)



['gre', 'gpa', 'prestige_2', 'prestige_3', 'prestige_4', 'intercept']


#### 4.2 Fit the model

In [222]:
print train_cols
lm = sm.Logit(data['admit'][:320],data[train_cols][:320])
result = lm.fit()
#from sklearn.linear_model import LogisticRegression
#lm = LogisticRegression()
#result = lm.fit(train_cols, data['admit'])

Index([u'gre', u'gpa', u'prestige_2', u'prestige_3', u'prestige_4',
       u'intercept'],
      dtype='object')
Optimization terminated successfully.
         Current function value: 0.561143
         Iterations 6


#### 4.3 Print the summary results

In [223]:

result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  320
Model:                          Logit   Df Residuals:                      314
Method:                           MLE   Df Model:                            5
Date:                Mon, 22 Aug 2016   Pseudo R-squ.:                 0.09651
Time:                        15:08:39   Log-Likelihood:                -179.57
converged:                       True   LL-Null:                       -198.75
                                        LLR p-value:                 3.188e-07
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
gre            0.0022      0.001      1.848      0.065        -0.000     0.005
gpa            0.9303      0.392      2.375      0.018         0.163     1.698
prestige_2    -0.6044      0.364     -1.659      0.097        -1.318     0.110
prestige_3    -1.5587      0.399     -3.904      0.000        -2.341    -0.776
prestige_4    -1.4807      0.447     -3.314      0.001        -2.356    -0.605
intercept     -4.3772      1.339     -3.270      0.001        -7.001    -1.754
==============================================================================
"""

In [224]:
#print zip(train_cols.columns,np.exp(lm.coef_[0]))



#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
       conf.columns = ['2.5%', '97.5%', 'OR']

In [225]:
print np.exp(result.params)

gre           1.002231
gpa           2.535169
prestige_2    0.546378
prestige_3    0.210405
prestige_4    0.227490
intercept     0.012560
dtype: float64


#### 4.5 Interpret the OR of Prestige_2

Answer: 

#### 4.6 Interpret the OR of GPA

Answer: 

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [226]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [227]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [228]:
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
print combos.head()

     gre       gpa  prestige  intercept
0  220.0  2.260000       1.0        1.0
1  220.0  2.260000       2.0        1.0
2  220.0  2.260000       3.0        1.0
3  220.0  2.260000       4.0        1.0
4  220.0  2.453333       1.0        1.0


In [229]:
# recreate the dummy variables

dummies2 = pd.get_dummies(combos['prestige'], prefix='prestige')
dummies2.columns = ['prestige_1', 'prestige_2', 'prestige_3', 'prestige_4']
combos = combos[['gre', 'gpa', 'intercept']].join(dummies2.ix[:, 'prestige_2':])
combos.head()


,gre,gpa,intercept,prestige_2,prestige_3,prestige_4
0,220.0,2.260000,1.0,0.0,0.0,0.0
1,220.0,2.260000,1.0,1.0,0.0,0.0
2,220.0,2.260000,1.0,0.0,1.0,0.0
3,220.0,2.260000,1.0,0.0,0.0,1.0
4,220.0,2.453333,1.0,0.0,0.0,0.0


#### 5.2 Make predictions on the enumerated dataset

In [238]:

combos['prediction'] = result.predict(combos[train_cols])
print combos.tail()


       gre       gpa  intercept  prestige_2  prestige_3  prestige_4  \
395  800.0  3.806667        1.0         0.0         0.0         1.0   
396  800.0  4.000000        1.0         0.0         0.0         0.0   
397  800.0  4.000000        1.0         1.0         0.0         0.0   
398  800.0  4.000000        1.0         0.0         1.0         0.0   
399  800.0  4.000000        1.0         0.0         0.0         1.0   

     prediction  
395    0.369518  
396    0.755140  
397    0.627562  
398    0.393528  
399    0.412307  


#### 5.3 Interpret findings for the last 4 observations

Answer: Odds are 75% if you are tier 1, 62% if you are tier 2, 39% if you are tier3, and 41% if you are tier 4.


## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.